The Gans Project by Balu, Marvin, Frederico and Joachim 

In this project, our E-scooter company is trying to predict  effective locations in Germany to place our scooters. For the purpose of this project we are trying to attract air travellers and  tourists to use our commodity as a means of transportation. As an experiment we are limiting ourselves to 16 of the largest air hubs in Germany which are also popular tourist destinations. 

In [2]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import time
#!pip install python-dotenv
#%load_ext dotenv
#%dotenv
import os
import pytz
from datetime import datetime
import mysql.connector
import sqlalchemy

The first task is to get the lat., long., population, country, countrycode and wiki_dataid for a list of cities in Germany and put it in a dataframe. This code is designed so that in future more cities maybe added.

In [30]:
#API_key = os.getenv("API_K_C")
 #gets the API key from .env
#define the list of cities

citd =['Frankfurt am Main']

#strategy is to first get the wikidataid and than collect the data from geoDB; openweather has no population data or at least I coul´nt see them 


API_key = "677319c679mshb16657ba32708bep1bcd54jsnec8935ebc2bf"
def demo(cities):
    cities_id = [] # initiate an empty id list
    dfList = []
    for city in cities:
        #retrieve the wikidataId
        time.sleep(2)
        url1 = 'https://en.wikipedia.org/wiki/{}'.format(city) #go to the wiki site of the city
        citem = requests.get(url1, 'html.parser') # get the html
        if BeautifulSoup(citem.content) != None:
            soup = BeautifulSoup(citem.content)# soup the content
        if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
            wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href'] #find the link that contains the wikidataid e.g. London https://www.wikidata.org/wiki/Q84
        #wl.append(wikidata_link)
        # \d+ is a regular expression and means one digit or more, the wiki data id consist of a Q followed by severaldigits
        #for group() in re see: https://www.tutorialspoint.com/What-is-the-groups-method-in-regular-expressions-in-Python
        city_id = re.search('Q\d+', wikidata_link).group()
        cities_id.append(city_id)
        #use the wikidataId to retrieve infrormation from geoDB
        url2 = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{}".format(city_id)
        headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
        }    
        response = requests.request("GET", url2, headers=headers)# gets a json-like string fromg geoDB containing the necessary informations
        print(response.json())
        cit_dic = {}#make a dictionary to retrieve the information
        cit_dic['City'] = response.json()['data']['name']
        cit_dic['Country'] = response.json()['data']['country']
        cit_dic['CountryCode'] = response.json()['data']['countryCode']
        cit_dic['WikiDataId'] = response.json()['data']['wikiDataId']
        cit_dic['Latitude'] = round(response.json()['data']['latitude'], 4)
        cit_dic['Longitude'] = round(response.json()['data']['longitude'], 4)
        cit_dic['Population'] = response.json()['data']['population']
        cit_dic['Timezone'] = response.json()['data']['timezone']
        
        dfList.append(cit_dic) #put it in a list
        df_demo = pd.DataFrame(dfList) # transform the list to df
        #print(dfList)
    return df_demo
demo(citd) #run the whole thing - good luck!

{'data': {'id': 3449552, 'wikiDataId': 'Q1794', 'type': 'CITY', 'city': 'Frankfurt am Main', 'name': 'Frankfurt am Main', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Hesse', 'regionCode': 'HE', 'elevationMeters': 112, 'latitude': 50.113611111, 'longitude': 8.679722222, 'population': 764104, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}


,City,Country,CountryCode,WikiDataId,Latitude,Longitude,Population,Timezone
0,Frankfurt am Main,Germany,DE,Q1794,50.1136,8.6797,764104,Europe__Berlin


Focussing on these 15 cities, we are collecting the weather forecast. This DF is subject to change as the weather information is updated as needed.

In [31]:
citd =['Frankfurt am Main']#alternatively only German cities

def weather(cities):
    API_key = "e1118d8a7ca0959abb3fbfac97f38322" #gets key from the env file

    tz = pytz.timezone('Europe/Berlin')# timezone needs to be imported and translated for the european cities (Europe_Berlin to Europe/Berlin), see cell below
    now = datetime.now().astimezone(tz)#for the timestamp time of sampling

    a = {'City': [], 'Country': [], 'ForecastTime': [], 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []} #usefull form of a dic, gives the right form when change to df, see cell below 

    for city in cities: #runs a loop over the cities
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        response = requests.get(url)
        json = response.json() # the first 'priciple' key for this json is 'list', which is not good practice in my op but must be used to loop over it
    # print(json['city']['name'])
    # print(type(json))

        for i in json['list']:# informations will be added to the dic a according to the json structure (see weimar cell above), if-else for rain does not work (is there or not but python cannot know that it could be there), therefore try except - appears to work but I´m not sure if clean
      #print(i)  
            a['City'].append(json['city']['name'])
            a['Country'].append(json['city']['country'])
            a['ForecastTime'].append(i['dt_txt'])
            a['Outlook'].append(i['weather'][0]['description'])
            a['Temperature'].append(i['main']['temp'])
            a['Clouds'].append(i['clouds']['all'])
            try:
                a['Rain'].append(i['rain']['3h'])
            except:
                a['Rain'].append('0')
            try:
                a['Snow'].append(i['snow']['3h'])
            except:
                a['Snow'].append('0')
            a['WindSpeed'].append(i['wind']['speed'])
            a['DateInformation'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
    b=pd.DataFrame(a) #turns a to datafram , see cell below
    
    return b
weather(citd)


,City,Country,ForecastTime,Outlook,Temperature,Clouds,Rain,Snow,WindSpeed,DateInformation
0,Frankfurt am Main,DE,2022-06-15 12:00:00,clear sky,26.08,8,0,0,0.91,15/06/2022 11:02:40
1,Frankfurt am Main,DE,2022-06-15 15:00:00,clear sky,28.16,7,0,0,0.77,15/06/2022 11:02:40
2,Frankfurt am Main,DE,2022-06-15 18:00:00,clear sky,26.88,6,0,0,2.16,15/06/2022 11:02:40
3,Frankfurt am Main,DE,2022-06-15 21:00:00,clear sky,21.00,5,0,0,1.53,15/06/2022 11:02:40
4,Frankfurt am Main,DE,2022-06-16 00:00:00,few clouds,17.99,23,0,0,2.19,15/06/2022 11:02:40
5,Frankfurt am Main,DE,2022-06-16 03:00:00,scattered clouds,16.32,37,0,0,2.25,15/06/2022 11:02:40
6,Frankfurt am Main,DE,2022-06-16 06:00:00,scattered clouds,19.78,47,0,0,3.07,15/06/2022 11:02:40
7,Frankfurt am Main,DE,2022-06-16 09:00:00,clear sky,24.81,4,0,0,3.17,15/06/2022 11:02:40
8,Frankfurt am Main,DE,2022-06-16 12:00:00,clear sky,27.92,7,0,0,3.12,15/06/2022 11:02:40
9,Frankfurt am Main,DE,2022-06-16 15:00:00,clear sky,28.27,10,0,0,4.18,15/06/2022 11:02:40


For the cities in our database we now get the details of the airports. 

In [48]:
#citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']
ac = ['EDDF']#, 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']
airport = []
API_KE = '677319c679mshb16657ba32708bep1bcd54jsnec8935ebc2bf'
for a in ac:
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{a}"

    querystring = {"withRunways":"true"}

    headers = {
        "X-RapidAPI-Key": API_KE,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    airport.append(response.json())

print(airport)
   # listCityShort[a]=response.json()['city']
    #print(listCityShort)

    

[{'icao': 'EDSB', 'iata': 'FKB', 'shortName': 'Karlsruhe', 'fullName': 'Baden-Baden, Karlsruhe Baden-Baden', 'municipalityName': 'Baden-Baden', 'location': {'lat': 48.7794, 'lon': 8.080499}, 'country': {'code': 'DE', 'name': 'Germany'}, 'continent': {'code': 'EU', 'name': 'Europe'}, 'timeZone': 'Europe/Berlin', 'urls': {'webSite': 'http://www.baden-airpark.de/startseite.html', 'wikipedia': 'https://en.wikipedia.org/wiki/Baden_Airpark', 'googleMaps': 'https://www.google.com/maps/@48.779399,8.080499,14z', 'flightRadar': 'https://www.flightradar24.com/48.78,8.08/14'}, 'runways': [{'name': '03', 'trueHdg': 31.7, 'length': {'meter': 3008.07, 'km': 3.008, 'mile': 1.869, 'nm': 1.624, 'feet': 9869.0}, 'width': {'meter': 60.05, 'km': 0.06, 'mile': 0.037, 'nm': 0.032, 'feet': 197.0}, 'isClosed': False, 'location': {'lat': 48.767845, 'lon': 8.069728}, 'surface': 'Asphalt', 'displacedThreshold': {'meter': 263.96, 'km': 0.264, 'mile': 0.164, 'nm': 0.143, 'feet': 866.0}, 'hasLighting': True}, {'name

In [37]:
#citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']
ac = ['EDDF']#, 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']

def airp(cities):
    airport = []
    API_KE = "1821798fe1mshc0aaf1be2c43cedp1289ecjsn546ec73b6483"
    a = {'icao': [], 'iata': [], 'airport_name': [], 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []}
    for a in ac:
        url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{a}"

        querystring = {"withRunways":"true"}

        headers = {
            "X-RapidAPI-Key": API_KE,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)
        air_json = response.json()
        print(air_json)
        for i in json['list']:

    #air = pd.json_normalize(airport)

    #air.drop(columns = ['fullName', 'runways', 'continent.code', 'urls.wikipedia', 'urls.googleMaps', 'urls.flightRadar', 'urls.twitter', 'location.lat', 'location.lon'], inplace = True)
    #air.columns = ['icao', 'iata', 'name', 'city', 'time_zone', 'country_code', 'country', 'continent', 'website']
    
    return air 
airp(ac)



{'icao': 'EDDF', 'iata': 'FRA', 'shortName': 'Frankfurt-am-Main', 'fullName': 'Frankfurt-am-Main', 'municipalityName': 'Frankfurt-am-Main', 'location': {'lat': 50.0264, 'lon': 8.543129}, 'country': {'code': 'DE', 'name': 'Germany'}, 'continent': {'code': 'EU', 'name': 'Europe'}, 'timeZone': 'Europe/Berlin', 'urls': {'webSite': 'http://www.frankfurt-airport.de/', 'wikipedia': 'https://en.wikipedia.org/wiki/Frankfurt_Airport', 'twitter': 'http://twitter.com/Airport_FRA', 'googleMaps': 'https://www.google.com/maps/@50.026401,8.543129,14z', 'flightRadar': 'https://www.flightradar24.com/50.03,8.54/14'}, 'runways': [{'name': '07C', 'trueHdg': 69.6, 'length': {'meter': 3994.1, 'km': 3.994, 'mile': 2.482, 'nm': 2.157, 'feet': 13104.0}, 'width': {'meter': 60.05, 'km': 0.06, 'mile': 0.037, 'nm': 0.032, 'feet': 197.0}, 'isClosed': False, 'location': {'lat': 50.032608, 'lon': 8.53463}, 'surface': 'Asphalt', 'displacedThreshold': {'meter': 0.0, 'km': 0.0, 'mile': 0.0, 'nm': 0.0, 'feet': 0.0}, 'hasL

NameError: name 'air' is not defined

In [36]:
air_json

NameError: name 'air_json' is not defined

In [29]:
airp(ac)

,icao,iata,shortName,fullName,municipalityName,timeZone,runways,location.lat,location.lon,country.code,country.name,continent.code,continent.name,urls.webSite,urls.wikipedia,urls.googleMaps,urls.flightRadar
0,EDSB,FKB,Karlsruhe,"Baden-Baden, Karlsruhe Baden-Baden",Baden-Baden,Europe/Berlin,"[{'name': '03', 'trueHdg': 31.7, 'length': {'m...",48.7794,8.080499,DE,Germany,EU,Europe,http://www.baden-airpark.de/startseite.html,https://en.wikipedia.org/wiki/Baden_Airpark,"https://www.google.com/maps/@48.779399,8.08049...","https://www.flightradar24.com/48.78,8.08/14"


In [19]:
test = airp(ac)
#airports = pd.DataFrame(test)

ValueError: Length mismatch: Expected axis has 10 elements, new values have 9 elements

In [18]:
test

NameError: name 'test' is not defined

In [16]:
airp(ac).shape()

ValueError: Length mismatch: Expected axis has 10 elements, new values have 9 elements

For all the airports in our list we can have the  details of the arriving flights. The most important detail being the flight times. The list may also optionally include the details of the flights origin as in future we can have a statistical analysis of the passengers who may use our scooters. This is also a changing DF and with queries we can find out the most and least busiest times of the day. This code is not ideal since it does not show the destination airport. Need to edit the codes.

In [14]:
import os

API_KE = '3cd15bf266msh2331a2a034ea490p1c96a2jsn828e9dce3a50' #os.getenv("API_C_F")
#print(API_KE)
cit = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']
things = ['number', 'status']
#def calculate(n):
 #   return n+2

def flights(cities):
    flight_details = []
    a = {'City': [], 'FlightNumber': [], 'FlightStatus': [], 'OriginAirport': [], 'ScheduledTime': [], 'AirLine': []}#, 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []}
    for city in cities:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{city}/2022-06-08T06:00/2022-06-08T17:00"
        querystring = {"direction":"Arrival","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}
        headers = {
           "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
           "X-RapidAPI-Key": API_KE
           }
        responses = requests.request("GET", url, headers=headers, params=querystring)
        json = responses.json()
        
        for i in json['arrivals']:
            a['City'] = city
            a['FlightNumber'].append(i['number'])
            a['FlightStatus'].append(i['status'])
            a['OriginAirport'].append(i['movement']['airport']['name'])
            a['ScheduledTime'].append(i['movement']['scheduledTimeLocal'])
            a['AirLine'].append(i['airline']['name'])
        c = pd.DataFrame(a)
    return c
flights(cit)



,City,FlightNumber,FlightStatus,OriginAirport,ScheduledTime,AirLine
0,EDDS,GFD 41,Approaching,Unknown,2022-06-08 11:49+02:00,GFD
1,EDDS,W6 6617,Unknown,Tirana,2022-06-08 13:30+02:00,Wizz Air
2,EDDS,FR 6117,Unknown,Sofia,2022-06-08 08:10+02:00,Ryanair
3,EDDS,FR 9211,Approaching,Girona,2022-06-08 15:50+02:00,Ryanair
4,EDDS,EW 6804,Unknown,Palma De Mallorca,2022-06-08 10:25+02:00,Eurowings
...,...,...,...,...,...,...
2480,EDDS,TK 1701,Arrived,Istanbul,2022-06-08 10:00+02:00,Turkish
2481,EDDS,TK 1703,Arrived,Istanbul,2022-06-08 14:25+02:00,Turkish
2482,EDDS,XC 5301,Arrived,Kayseri,2022-06-08 06:50+02:00,Corendon Air
2483,EDDS,XQ 152,Arrived,Antalya,2022-06-08 15:35+02:00,SunExpress


In [9]:
from IPython.display import JSON
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDSB/2022-06-08T06:00/2022-06-08T17:00"
querystring = {"direction":"Arrival","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}
headers = {
     "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
     "X-RapidAPI-Key": API_KE
     }
responses = requests.request("GET", url, headers=headers, params=querystring)
#print(type(responses.json()))
draft = responses.json()
JSON(draft)

<IPython.core.display.JSON object>

In [23]:
demo(citd)

{'data': {'id': 25706, 'wikiDataId': 'Q4100', 'type': 'CITY', 'city': 'Baden-Baden', 'name': 'Baden-Baden', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Baden-Württemberg', 'regionCode': 'BW', 'elevationMeters': 181, 'latitude': 48.761944444, 'longitude': 8.240833333, 'population': 55449, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3453309, 'wikiDataId': 'Q64', 'type': 'CITY', 'city': 'Berlin', 'name': 'Berlin', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Berlin', 'regionCode': 'BE', 'elevationMeters': None, 'latitude': 52.516666666, 'longitude': 13.383333333, 'population': 3664088, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3449715, 'wikiDataId': 'Q586', 'type': 'CITY', 'city': 'Bonn', 'name': 'Bonn', 'country': 'Germany', 'countryCode': 'DE', 'region': 'North Rhine-Westphalia', 'regionCode': 'NW', 'elevationMeters': None, 'latitude': 50.733888888

,City,Country,CountryCode,WikiDataId,Latitude,Longitude,Population,Timezone
0,Baden-Baden,Germany,DE,Q4100,48.7619,8.2408,55449,Europe__Berlin
1,Berlin,Germany,DE,Q64,52.5167,13.3833,3664088,Europe__Berlin
2,Bonn,Germany,DE,Q586,50.7339,7.0997,330579,Europe__Berlin
3,Bremen,Germany,DE,Q24879,53.0769,8.8089,566573,Europe__Berlin
4,Dresden,Germany,DE,Q1731,51.0833,13.7666,554907,Europe__Berlin
5,Dortmund,Germany,DE,Q1295,51.5139,7.4653,587696,Europe__Berlin
6,Düsseldorf,Germany,DE,Q1718,51.2311,6.7724,645923,Europe__Berlin
7,Essen,Germany,DE,Q2066,51.4508,7.0131,582415,Europe__Berlin
8,Frankfurt am Main,Germany,DE,Q1794,50.1136,8.6797,764104,Europe__Berlin
9,Hamburg,Germany,DE,Q1055,53.5500,10.0000,1852478,Europe__Berlin


In [ ]:
weather(citd)

In [ ]:
flights(cit)

KeyError: 'arrivals'

In [30]:
airp(ac)

KeyError: "['fullName' 'runways' 'continent.code' 'urls.wikipedia' 'urls.googleMaps'\n 'urls.flightRadar' 'urls.twitter' 'location.lat' 'location.lon'] not found in axis"

After completing all the DFs, they are exported to an already created SQL Schema. Note: No tables were created in MySQL workbench all tables are created from here. Once created go to sql to create Primary and Foreign Keys. Also the flights DF is not exported as the city column is missing and I have ran out of API calls.

Please feel free to alter the codes in anyway that seem fit.

In [3]:

cnx = mysql.connector.connect(
    user='admin',
    password='!Kochumol20', #type your root password here
    host='gans.czi7af4kh5fl.us-east-1.rds.amazonaws.com', # to connect to your local instance
    database='gans' #type the name of the database you want to use here
)
 
# connect to database
cursor = cnx.cursor()
schema="gans"
host="gans.czi7af4kh5fl.us-east-1.rds.amazonaws.com"
user="admin"
password="!Kochumol20"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [8]:
#!pip install pymysql
#weather(citd).to_sql('weather', 
               #if_exists='append', 
               #con=con, 
               #index=False)
airp(ac).to_sql('airports', 
               if_exists='append', 
               con=con, 
               index=False)
#demo(citd).to_sql('cities', 
               #if_exists='append', 
               #con=con, 
               #index=False)
#flights(cit).to_sql('flights', 
               #if_exists='append', 
               #con=con, 
               #index=False)